In [1]:
import requests
import pandas as pd
from pytz import timezone
import datetime

In [11]:
api_key = "pul79sgtbk000000"

api_url = "https://freeserv.dukascopy.com/2.0/?path=api/historicalPrices"

instrument = "2024"
timeFrame = "1min"

In [12]:
response = requests.get(api_url, params={'key': api_key,
                                             'instrument': instrument,
                                             'timeFrame': timeFrame,
                                             'count': 5000,
                                             'offerSide': 'B'})

if response.status_code == 200:
    response_data = response.json()
    bid_data = response_data['candles']
else:
    print(f"Error in API request {request_number + 1}: {response.status_code}")

response = requests.get(api_url, params={'key': api_key,
                                             'instrument': instrument,
                                             'timeFrame': timeFrame,
                                             'count': 5000,
                                             'offerSide': 'A'})

if response.status_code == 200:
    response_data = response.json()
    ask_data = response_data['candles']
else:
    print(f"Error in API request {request_number + 1}: {response.status_code}")

bid_df = pd.DataFrame(bid_data)
bid_df.index = pd.to_datetime(bid_df.timestamp, unit='ms')

ask_df = pd.DataFrame(ask_data)
ask_df.index = pd.to_datetime(ask_df.timestamp, unit='ms')
ask_df = ask_df.drop(['timestamp'], axis = 1)

df = bid_df.join(ask_df)

df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize('UTC')
london = timezone('Europe/London')

df.index = df.index.tz_convert(london)

df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')

df.index = pd.to_datetime(df.index)
df.index.names = ['date']

#Date Setup
current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month
current_day = datetime.datetime.now().day

#Current Data
current_file_loc = 's3://fx-data-raw/gbpjpy/oneminute/'+str(current_year)+'_'+str(current_month).zfill(2)+'.parquet'
try:
    current_data = pd.read_parquet(current_file_loc)
except:
    current_data = None

if current_data is None:
    combined_df = df 
else: 
    combined_df = current_data.append(df)
    combined_df = combined_df[~combined_df.index.duplicated(keep='first')]

combined_df.to_parquet(current_file_loc)

In [2]:
#Date Setup
current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month
current_day = datetime.datetime.now().day

#Current Data
current_file_loc = 's3://fx-data-raw/gbpjpy/oneminute/'+str(current_year)+'_'+str(current_month).zfill(2)+'.parquet'
try:
    current_data = pd.read_parquet(current_file_loc)
except:
    current_data = None

In [3]:
current_data

,timestamp,bid_open,bid_high,bid_low,bid_close,bid_volume,ask_open,ask_high,ask_low,ask_close,ask_volume
date,,,,,,,,,,,
2023-05-01 00:00:00,1682895600000,171.025,171.076,171.020,171.069,252.46,171.058,171.116,171.054,171.112,224.93
2023-05-01 00:01:00,1682895660000,171.068,171.104,171.062,171.104,263.18,171.113,171.141,171.105,171.141,230.42
2023-05-01 00:02:00,1682895720000,171.106,171.108,171.075,171.078,209.88,171.151,171.155,171.125,171.126,174.21
2023-05-01 00:03:00,1682895780000,171.077,171.123,171.072,171.114,150.90,171.125,171.165,171.119,171.161,125.95
2023-05-01 00:04:00,1682895840000,171.114,171.148,171.110,171.143,141.82,171.162,171.189,171.157,171.187,121.55
...,...,...,...,...,...,...,...,...,...,...,...
2023-05-18 10:29:00,1684402140000,171.323,171.365,171.313,171.354,163.60,171.342,171.384,171.333,171.373,160.59
2023-05-18 10:30:00,1684402200000,171.357,171.373,171.320,171.325,134.71,171.374,171.391,171.338,171.344,127.27
2023-05-18 10:31:00,1684402260000,171.326,171.347,171.324,171.341,92.55,171.345,171.364,171.344,171.358,123.62
